In [1]:
import pandas as pd 
import numpy as np 


In [3]:
history = pd.read_json('MyData/StreamingHistory0.json')


In [34]:
history.head()

,endTime,artistName,trackName,msPlayed
0,2020-12-30 15:23,Dastic,With Me,89517
1,2020-12-30 17:26,Philosophize This!,Episode #001 ... Presocratic Philosophy - Ionian,2188097
2,2020-12-31 07:09,Dastic,With Me,2530
3,2020-12-31 07:12,Mark Sixma,Meet Again,186307
4,2020-12-31 07:15,Jubël,Dancing In The Moonlight (feat. NEIMY) - Tiëst...,147146


# Getting features of played songs


### Flow 

1. Get the id of the songs through Artist name and track name 
2. Get track features from id 

In [15]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time
import datetime

In [16]:
SPOTIPY_CLIENT_ID=''
SPOTIPY_CLIENT_SECRET=''
SPOTIPY_REDIRECT_URI='http://127.0.0.1:9090/'
SCOPE = "user-top-read"

In [17]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID, 
                                               client_secret=SPOTIPY_CLIENT_SECRET, 
                                               redirect_uri=SPOTIPY_REDIRECT_URI, 
                                               scope=SCOPE))

## Getting track id 

In [18]:
unique_songs = history.groupby(['trackName', 'artistName']).size().reset_index(name='Freq') # extracting unique songs 

In [19]:
unique_songs = unique_songs.sort_values(by = 'Freq',ascending = False).reset_index(drop =True)

In [20]:
unique_songs['id'] = np.nan

In [23]:
def get_song_id(artist,track):
    try:
        q = 'artist:{} track: {}'.format(artist, track)
        results = sp.search(q=q, limit=1, type='track')
        id = results['tracks']['items'][0]['id']
        return id
    except:
        return

In [24]:
for i in unique_songs.index.to_list():
    artist =unique_songs.iloc[i,1]
    track = unique_songs.iloc[i,0]
    track_id = get_song_id(artist,track)
    unique_songs['id'][i] = track_id

<ipython-input-24-09d207938b04>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_songs['id'][i] = track_id
C:\Users\Lohiya\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [25]:
unique_songs.dropna(inplace =True) 

In [26]:
unique_songs[66:85]

,trackName,artistName,Freq,id
66,You Right,Doja Cat,23,0k4d5YPDr1r7FX77VdqWez
67,Home with You,Madison Beer,23,0iwsQWgtjSq2kUXuZwTDAL
68,Kissing Strangers,DNCE,22,1NkwZ9TnIs8js6G0M6M7RU
69,Bad Blood,Taylor Swift,22,0TvQLMecTE8utzoNmvXRbK
70,Break My Face,AJR,22,60eAbsb4svFn3d62wfwEUe
71,Remember The Name (feat. Eminem & 50 Cent),Ed Sheeran,22,0AtP8EkGPn6SwxKDaUuXec
72,Gorgeous,Taylor Swift,22,1ZY1PqizIl78geGM4xWlEA
73,No One Compares To You,Jack & Jack,22,4cZ3UsiKd1kUQIaq4BFIj2
74,What A Man Gotta Do,Jonas Brothers,22,4wNIkl5XGiAACjFBlDWuSd
75,Drag Me Down,One Direction,22,2K87XMYnUMqLcX3zvtAF4G


## Getting track fearture 

In [27]:
clms = ['trackName','artistName','id','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
track_features = pd.DataFrame(columns=clms)

In [28]:
track_features

,trackName,artistName,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [29]:
def get_feature(track_features,trackName,artistName,id):
    
    clms = ['trackName','artistName','id','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
    track = {'trackName':trackName,'artistName':artistName}
    meta = sp.audio_features(id)
    track.update(meta[0])
    tempDf = pd.DataFrame.from_dict([track])[clms]
    track_features = pd.concat([track_features,tempDf],ignore_index=True)
    return track_features
    

In [30]:
for i in unique_songs.index[1:]:
    trackName = unique_songs['trackName'][i]
    artistName = unique_songs['artistName'][i]

    id = unique_songs['id'][i]
    track_features = get_feature(track_features,trackName,artistName,id)

In [31]:
track_features.isna().sum()

trackName           0
artistName          0
id                  0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64

## exporting the data 

In [32]:
track_features.to_csv('MyData/track_features.csv')

In [33]:
unique_songs.to_csv('MyData/unique_songs.csv')